In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pickle
import random
from sklearn.metrics import accuracy_score,f1_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm
import sys
import os
import warnings
if not sys.warnoptions:
	warnings.simplefilter("ignore")							#1
	os.environ["PYTHONWARNINGS"] = "ignore"					#2

	warnings.filterwarnings('ignore', 'Solver terminated early.*')		#3
	# warnings.filterwarnings('ignore', category="ConvergenceWarning")	#4
from datetime import datetime

In [3]:
m = datetime.now().month
d = datetime.now().day

In [3]:
# 데이터 읽기
train = pd.read_csv('./2023_ML_프로젝트_데이터/Train.csv')
test = pd.read_csv('./2023_ML_프로젝트_데이터/Test.csv')
# index drop
train = train.drop('Index',axis=1)
test = test.drop('Index',axis=1)
# train용 데이터의 독립변수와 종속변수의 분리
X = train.loc[:,:'X999']
y = train.iloc[:,-1]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)
#svm parameter tuning
param_dist = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]
}

gs = GridSearchCV(SVC(), param_dist, cv=5,scoring='f1_macro')
gs.fit(X_train,y_train)

print('test f1 : ', gs.score(X_test, y_test))
print('최적의 parameters : ', gs.best_params_)

In [82]:
# score 를 보기 위해 split, 제출용은 Full Train으로 제작할 것
def test_smote_model():
    # score 를 보기 위해 split, 제출용은 Full Train으로 제작할 것
    m = datetime.now().month
    d = datetime.now().day

    i = random.randint(1,1000)
    j = random.randint(1,1000)
    k = random.randint(1,1000)
    l = random.randint(1,1000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=j)
    smote = SMOTE(random_state=i)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # 앙상블 방식
    classifiers = [
        RandomForestClassifier(n_estimators=30,random_state=k),
        LogisticRegression(max_iter=20),
        GradientBoostingClassifier(n_estimators=30),
        DecisionTreeClassifier(max_depth=20),
        GaussianNB(),
        # SVC(kernel="rbf")
        # AdaBoostClassifier(n_estimators = 30),
        XGBClassifier(objective='binary:logistic'),
        MLPClassifier(random_state=l)
    ]
    result = random.sample(classifiers,3)
    # clf_dic = {'RandomForestClassifier(n_estimators=30,random_state={})'.format(k):'RandomForest',
    #                     'LogisticRegression(max_iter=20)':'LogiReg',
    #                     'GradientBoostingClassifier(n_estimators=30)':'GBC',
    #                     'KNeighborsClassifier()':'KNN',
    #                     'DecisionTreeClassifier(max_depth=20)':'DT',
    #                     'SVC(kernel="rbf")':'SVC'                        
    #     }
    a,b,c = result[0],result[1],result[2]
    # a_,b_,c_ = clf_dic[str(a)],clf_dic[str(b)],clf_dic[str(c)]
    weights1,weights2,weights3 = random.randint(1,6),random.randint(1,6),random.randint(1,6)
    model = VotingClassifier(estimators=[('1',a),('2',b),('3',c)],voting = 'soft',weights=[weights1,weights2,weights3])
    ensemble_train = model.fit(X_train_resampled,y_train_resampled)

    ensemble_pred = model.predict(X_test)
    score = f1_score(y_test,ensemble_pred,pos_label='positive')
    if round(score,2) >= 0.52:
        with open('model/smote_test_{}_{}월_{}일_({}_{})({},{},{}).pkl'.format(round(score,4),m,d,i,j,a,b,c),'wb') as f:
            pickle.dump(model,f)
    return round(score,4)
    # print("Classificcation Report: \n{}".format(classification_report(y_test,ensemble_pred)))


In [83]:
for i in tqdm(range(5)):
    a = test_smote_model()
    print(a)


 20%|██        | 1/5 [00:48<03:13, 48.45s/it]

0.262


 40%|████      | 2/5 [00:59<01:20, 26.74s/it]

0.341


 60%|██████    | 3/5 [01:44<01:09, 34.84s/it]

0.2656


 80%|████████  | 4/5 [02:42<00:44, 44.16s/it]

0.4216


100%|██████████| 5/5 [02:54<00:00, 34.89s/it]

0.2812


In [5]:
with open('C:/Users/alseh/Desktop/jeonmincheol/2023/2023-1/binary_classification/model/smote_test_0.5381_6월_9일_(LogisticRegression(max_iter=20),GradientBoostingClassifier(n_estimators=30),MLPClassifier(random_state=124)).pkl','rb') as f:
    model = pickle.load(f)

In [7]:
smote = SMOTE(random_state=87)
X_resampled, y_resampled = smote.fit_resample(X, y)
model.fit(X_resampled,y_resampled)

VotingClassifier(estimators=[('1', LogisticRegression(max_iter=20)),
                             ('2', GradientBoostingClassifier(n_estimators=30)),
                             ('3', MLPClassifier(random_state=124))],
                 voting='soft', weights=[2, 1, 6])

In [11]:
# model test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)
smote = SMOTE(random_state=23)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# # 앙상블 방식
# classifiers = {}
# classifiers1 = RandomForestClassifier(n_estimators=30)
# classifiers2 = LogisticRegression(max_iter=20)
# # classifiers2 = GradientBoostingClassifier(n_estimators=30)
# classifiers3 = KNeighborsClassifier()
# # classifiers3 = AdaBoostClassifier(n_estimators = 30)
# # ensemble_train = model.fit(X_train_resampled,y_train_resampled)

# ensemble_pred = model.predict(X_test)
# score = f1_score(y_test,ensemble_pred,pos_label='positive')

In [8]:
ensemble_pred_for_csv = model.predict(test.loc[:,:'X999'])

print(f'Type : {type(ensemble_pred_for_csv)} | Length : {len(ensemble_pred_for_csv)}')

import numpy as np

np.savetxt('./버킷햇_0609.csv', ensemble_pred_for_csv, delimiter=',',fmt='%s')

Type : <class 'numpy.ndarray'> | Length : 3500


In [13]:
from sklearn.metrics import f1_score
# print(classification_report(y_test,ensemble_pred))
# print(accuracy_score(y_test,ensemble_pred))
print(f1_score(y_test,ensemble_pred,pos_label='negative',average=None))

ValueError: Found input variables with inconsistent numbers of samples: [1000, 3500]

In [ ]:
# 제출에 사용할 결과
ensemble_pred_for_csv = ensemble_train.predict(test.loc[:,:'X999'])

In [31]:
#logistice 버전
# Assuming you have your feature data in X and target data in y
# X is a 2D array with shape (5000, num_features)
# y is a 1D array with shape (5000,)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train a classification model (e.g., logistic regression)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = model.predict(X_test)
# Calculate the F1-score
# f1score = classification_report(y_test, y_pred)
print("Classificcation Report: \n{}".format(classification_report(y_test, y_pred)))
# print(f1score)


Classificcation Report: 
              precision    recall  f1-score   support

    negative       0.97      0.89      0.93       925
    positive       0.33      0.67      0.44        75

    accuracy                           0.88      1000
   macro avg       0.65      0.78      0.69      1000
weighted avg       0.92      0.88      0.89      1000



c:\Users\alseh\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
type(y_pred)

numpy.ndarray

In [11]:
test_ = test.loc[:,'X0':'X999']

In [12]:
type(y_pred)

numpy.ndarray

In [13]:
y_pred_ = model.predict(test_)
print(f'Type : {type(y_pred_)} | Length : {len(y_pred_)}')

Type : <class 'numpy.ndarray'> | Length : 3500


In [14]:
import numpy as np

np.savetxt('./버킷햇_0606.csv', y_pred_, delimiter=',',fmt='%s')

In [74]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# svc 버전
# Assuming you have your feature data in X and target data in y
# X is a 2D array with shape (5000, num_features)
# y is a 1D array with shape (5000,)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train an SVM model
model = SVC()
model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the F1-score
# f1score = classification_report(y_test, y_pred)
print("Classificcation Report: \n{}".format(classification_report(y_test, y_pred)))
# print(f1score)


Classificcation Report: 
              precision    recall  f1-score   support

    negative       0.95      0.86      0.90       902
    positive       0.30      0.56      0.39        98

    accuracy                           0.83      1000
   macro avg       0.62      0.71      0.64      1000
weighted avg       0.88      0.83      0.85      1000



In [33]:
len(y_pred)

3500

In [9]:
train.target.value_counts()

negative    4564
positive     436
Name: target, dtype: int64